In [1]:
# AutoGluon - Predicción de ventas (tn) por producto para febrero 2020

In [2]:
## 1. Importar librerías necesarias
import pandas as pd

In [3]:
%pip install autogluon.timeseries
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame

Note: you may need to restart the kernel to use updated packages.


c:\Users\fraco\anaconda3\envs\agluon39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
## 2. Cargar datasets
df_sellin = pd.read_csv("sell-in.txt", sep="\t")
df_productos = pd.read_csv("tb_productos.txt", sep="\t")

In [5]:
# Leer lista de productos a predecir
with open("product_id_apredecir201912.TXT", "r") as f:
    product_ids = [int(line.strip()) for line in f if line.strip().isdigit()]

In [6]:
## 3. Preprocesamiento
# Convertir periodo a datetime
df_sellin['timestamp'] = pd.to_datetime(df_sellin['periodo'], format='%Y%m')

In [7]:
# Filtrar hasta dic 2019 y productos requeridos
df_filtered = df_sellin[
    (df_sellin['timestamp'] <= '2019-12-01') &
    (df_sellin['product_id'].isin(product_ids))
]

In [8]:
# Agregar tn por periodo, cliente y producto
df_grouped = df_filtered.groupby(['timestamp', 'customer_id', 'product_id'], as_index=False)['tn'].sum()

In [9]:
# Agregar tn total por periodo y producto
df_monthly_product = df_grouped.groupby(['timestamp', 'product_id'], as_index=False)['tn'].sum()

In [10]:
# Agregar columna 'item_id' para AutoGluon
df_monthly_product['item_id'] = df_monthly_product['product_id']

In [11]:
## 4. Crear TimeSeriesDataFrame
ts_data = TimeSeriesDataFrame.from_data_frame(
    df_monthly_product,
    id_column='item_id',
    timestamp_column='timestamp'
)

In [12]:
print(ts_data.loc[20001])

            product_id          tn
timestamp                         
2017-01-01       20001   934.77222
2017-02-01       20001   798.01620
2017-03-01       20001  1303.35771
2017-04-01       20001  1069.96130
2017-05-01       20001  1502.20132
2017-06-01       20001  1520.06539
2017-07-01       20001  1030.67391
2017-08-01       20001  1267.39462
2017-09-01       20001  1316.94604
2017-10-01       20001  1439.75563
2017-11-01       20001  1580.47401
2017-12-01       20001  1049.38860
2018-01-01       20001  1169.07532
2018-02-01       20001  1043.76470
2018-03-01       20001  1856.83534
2018-04-01       20001  1251.28462
2018-05-01       20001  1293.89788
2018-06-01       20001  1150.79169
2018-07-01       20001  1470.41009
2018-08-01       20001  1800.96168
2018-09-01       20001  1438.67455
2018-10-01       20001  2295.19832
2018-11-01       20001  1813.01511
2018-12-01       20001  1486.68669
2019-01-01       20001  1275.77351
2019-02-01       20001  1259.09363
2019-03-01       200

In [13]:
# Completar valores faltantes
ts_data = ts_data.fill_missing_values()

In [14]:
predictor = TimeSeriesPredictor(
    prediction_length=2,
    target='tn',
    freq='MS'  # Frecuencia mensual (Month Start), 
)

In [15]:
predictor.fit(ts_data, num_val_windows=2, time_limit=60*60)

Beginning AutoGluon training... Time limit = 3600s
AutoGluon will save models to 'c:\Users\fraco\Documents\Github\Labo 3\AutogluonModels\ag-20250701_233133'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.9.23
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
GPU Count:          0
Memory Avail:       36.38 GB / 63.69 GB (57.1%)
Disk Space Avail:   592.71 GB / 909.94 GB (65.1%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'MS',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 2,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'tn',
 'time_limit': 3600,
 'verbosity': 2}

train_data with frequency 'IRREG' has been resampled to frequency 'MS'.
Provided train

In [16]:
print(ts_data.num_items)
print(ts_data.freq)

780
MS


In [17]:
forecast = predictor.predict(ts_data)

data with frequency 'IRREG' has been resampled to frequency 'MS'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


In [18]:
print(forecast.columns)

Index(['mean', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9'], dtype='object')


In [19]:
forecast_mean = forecast['mean'].reset_index()
print(forecast_mean.columns)

Index(['item_id', 'timestamp', 'mean'], dtype='object')


In [20]:
# Tomar solo item_id y la predicción 'mean'
resultado = forecast['mean'].reset_index()[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']

# Filtrar solo febrero 2020
resultado = forecast['mean'].reset_index()
resultado = resultado[resultado['timestamp'] == '2020-02-01']

# Renombrar columnas
resultado = resultado[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']

# Guardar a CSV
resultado.to_csv("predicciones_febrero2020_26-06-25_5.csv", index=False)
resultado.head()

,product_id,tn
1,20001,1305.667077
3,20002,1063.011962
5,20003,701.688985
7,20004,529.550740
9,20005,509.063156
